In [ ]:
import requests
import json
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  # This loads the .env file into the environment
ant_key = os.getenv("ANT_KEY")

import anthropic

client = anthropic.Anthropic()

message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=1000,
    temperature=1,
    system="You are a world-class poet. Respond only with short poems.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Why is the ocean salty?"
                }
            ]
        }
    ]
)


In [30]:
from typing import Optional

def call_claude(system, messages, think_budget:Optional[int]=None, max_tokens=2000, temp=1):
    """Thinking requires a minimum budget of 1,024 tokens and counts towards your max_tokens limit."""
    if think_budget is None:
        message = client.messages.create(
            model="claude-3-7-sonnet-20250219",
            max_tokens=max_tokens,
            thinking={"type": "disabled"},
            temperature=temp,
            system=system,
            messages=messages
        )
    else:
        message = client.messages.create(
            model="claude-3-7-sonnet-20250219",
            max_tokens=max_tokens,
            thinking={
                "type": "enabled",
                "budget_tokens": think_budget,
            },
            temperature=temp,
            system=system,
            messages=messages
        )
    return message.content

system="You are a world-class poet. Respond only with short poems."
messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Why is the ocean salty?"
                }
            ]
        }
    ]

response = call_claude(system, messages, think_budget=1024)

In [33]:
print(response[0].thinking)

This question is asking for a poem about why the ocean is salty. I'll create a short, evocative poem that touches on this natural phenomenon creatively. I can reference either the scientific explanation (minerals from the land washing into the sea) or incorporate mythological/emotional elements, or blend both approaches.

Some ideas:
- The ocean's tears collected over eons
- Earth's ancient minerals dissolving into water
- The relationship between land and sea
- Evaporation leaving salt behind
- Could incorporate mythology or personification

I'll write a short poem that's thoughtful but accessible, with some vivid imagery.


In [ ]:
import requests
import json
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
api_key=os.getenv("OR_API_KEY")
print(api_key)

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=api_key,
)

completion = client.chat.completions.create(
  model="anthropic/claude-3.7-sonnet",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What is in this image?"
        },
      ]
    }
  ]
)
print(completion.choices[0].message.content)

In [ ]:
import random

class BattleshipGame:
    def __init__(self, board_size=5):
        self.board_size = board_size
        # Define ship names and their lengths.
        self.ship_info = {
            # "Carrier": 5,
            # "Battleship": 4,
            "Cruiser": 3,
            # "Submarine": 3,
            # "Destroyer": 2
        }
        # Initialize two players with their boards and ship placements.
        self.players = [self._initialize_player_board(), self._initialize_player_board()]
        self.current_player = 0  # Index of the player whose turn it is

    def _initialize_player_board(self):
        """Creates an empty board and randomly places ships on it."""
        # Create an empty board (using '~' for water)
        board = [['~' for _ in range(self.board_size)] for _ in range(self.board_size)]
        ships = {}
        # Place each ship randomly
        for ship, size in self.ship_info.items():
            placed = False
            while not placed:
                orientation = random.choice(['H', 'V'])
                if orientation == 'H':
                    row = random.randint(0, self.board_size - 1)
                    col = random.randint(0, self.board_size - size)
                    # Check if space is free horizontally
                    if all(board[row][col + i] == '~' for i in range(size)):
                        for i in range(size):
                            board[row][col + i] = ship[0]  # Mark with first letter of the ship
                        ships[ship] = {'coords': [(row, col + i) for i in range(size)], 'hits': set()}
                        placed = True
                else:  # Vertical orientation
                    row = random.randint(0, self.board_size - size)
                    col = random.randint(0, self.board_size - 1)
                    # Check if space is free vertically
                    if all(board[row + i][col] == '~' for i in range(size)):
                        for i in range(size):
                            board[row + i][col] = ship[0]
                        ships[ship] = {'coords': [(row + i, col) for i in range(size)], 'hits': set()}
                        placed = True
        return {'board': board, 'ships': ships, 'guesses': set()}

    def print_board(self, player_index, print=True, reveal_ships=False):
        """
        Prints the board for the given player.
        If reveal_ships is False, all ship markers are hidden (displayed as water).
        """
        board = self.players[player_index]['board']
        display_board = []
        for row in board:
            display_row = []
            for cell in row:
                # Hide ship placements if not revealing
                if cell != '~' and not reveal_ships and cell not in ['X', 'O']:
                    display_row.append('~')
                else:
                    display_row.append(cell)
            display_board.append(display_row)
        # Print header with column numbers
        board_str = "  " + " ".join(str(i) for i in range(self.board_size)) + "\n"
        for idx, row in enumerate(display_board):
            board_str += f"{idx} " + " ".join(row) + "\n"
        if print:
            print(board_str)
    
    def make_prompt(self, player_index):
        """Returns a system prompt and a conversation dictionary"""
        board_str = self.print_board(1-player_index, print=False, reveal_ships=False)
        system = "You are playing the game of Battleship against an opponent. Your goal is to sink all of your opponent's ships."
        prompt = "Here is your observation of your opponent's board. X represents a hit, O represents a miss, and ~ represents unknown cells.\n"
        prompt += board_str
        prompt += "Please "

        return system, message


    def take_turn(self, attacker_index, target_index, row, col):
        """
        The attacker fires at (row, col) on the target's board.
        Returns True if it was a hit, False if a miss.
        """
        target = self.players[target_index]
        # Prevent firing at the same cell twice.
        if (row, col) in target['guesses']:
            print("You have already fired at that position. Try again.")
            return None  # Indicate invalid move
        target['guesses'].add((row, col))
        cell = target['board'][row][col]
        if cell != '~' and cell not in ['X', 'O']:
            # It’s a hit.
            print("Hit!")
            # Determine which ship was hit.
            for ship, data in target['ships'].items():
                if (row, col) in data['coords']:
                    data['hits'].add((row, col))
                    if len(data['hits']) == len(data['coords']):
                        print(f"You sunk the {ship}!")
                    break
            # Mark the hit on the board.
            target['board'][row][col] = 'X'
            return True
        else:
            print("Miss!")
            target['board'][row][col] = 'O'
            return False

    def check_winner(self):
        """
        Checks both players' boards to determine if one has lost all ships.
        Returns the index of the defeated player if found, or None otherwise.
        """
        for idx, player in enumerate(self.players):
            all_sunk = True
            for ship, data in player['ships'].items():
                if len(data['hits']) != len(data['coords']):
                    all_sunk = False
                    break
            if all_sunk:
                return idx  # This player's ships are all sunk.
        return None

    def play(self):
        """Main loop to run the game until one player wins."""
        print("Starting Battleship Game!\n")
        while True:
            attacker = self.current_player
            defender = 1 - self.current_player
            print(f"Player {attacker + 1}'s turn to attack Player {defender + 1}'s board.")
            # Show the attacker's view of the defender's board (ships hidden)
            self.print_board(defender, reveal_ships=False)
            
            # Get valid input for row and column.
            while True:
                try:
                    row = int(input("Enter row to fire: "))
                    col = int(input("Enter column to fire: "))
                    if 0 <= row < self.board_size and 0 <= col < self.board_size:
                        break
                    else:
                        print("Coordinates out of range. Please try again.")
                except ValueError:
                    print("Invalid input. Please enter integer values.")
            
            # Process the move; if the cell was already guessed, ask for another input.
            result = self.take_turn(attacker, defender, row, col)
            if result is None:
                continue  # Invalid move; let the same player try again.

            # Check if the defender has lost all ships.
            loser = self.check_winner()
            if loser is not None:
                winner = 1 - loser
                print(f"\nPlayer {winner + 1} wins! All of Player {loser + 1}'s ships have been sunk.")
                print("Final boards:")
                print("Player 1's board:")
                self.print_board(0, reveal_ships=True)
                print("Player 2's board:")
                self.print_board(1, reveal_ships=True)
                break
            
            # Switch turns.
            self.current_player = defender

if __name__ == "__main__":
    game = BattleshipGame()
    game.play()

Starting Battleship Game!

Player 1's turn to attack Player 2's board.
  0 1 2 3 4
0 ~ ~ ~ ~ ~
1 ~ ~ ~ ~ ~
2 ~ ~ ~ ~ ~
3 ~ ~ ~ ~ ~
4 ~ ~ ~ ~ ~

Miss!
Player 2's turn to attack Player 1's board.
  0 1 2 3 4
0 ~ ~ ~ ~ ~
1 ~ ~ ~ ~ ~
2 ~ ~ ~ ~ ~
3 ~ ~ ~ ~ ~
4 ~ ~ ~ ~ ~

Miss!
Player 1's turn to attack Player 2's board.
  0 1 2 3 4
0 ~ ~ ~ ~ ~
1 ~ ~ ~ ~ ~
2 ~ ~ O ~ ~
3 ~ ~ ~ ~ ~
4 ~ ~ ~ ~ ~

Miss!
Player 2's turn to attack Player 1's board.
  0 1 2 3 4
0 ~ ~ ~ ~ ~
1 ~ ~ ~ ~ ~
2 ~ ~ O ~ ~
3 ~ ~ ~ ~ ~
4 ~ ~ ~ ~ ~

Hit!
Player 1's turn to attack Player 2's board.
  0 1 2 3 4
0 ~ ~ ~ ~ ~
1 ~ O ~ ~ ~
2 ~ ~ O ~ ~
3 ~ ~ ~ ~ ~
4 ~ ~ ~ ~ ~

Miss!
Player 2's turn to attack Player 1's board.
  0 1 2 3 4
0 ~ ~ ~ ~ ~
1 ~ X ~ ~ ~
2 ~ ~ O ~ ~
3 ~ ~ ~ ~ ~
4 ~ ~ ~ ~ ~

Hit!
Player 1's turn to attack Player 2's board.
  0 1 2 3 4
0 ~ ~ ~ ~ ~
1 ~ O ~ ~ ~
2 ~ ~ O ~ ~
3 ~ ~ ~ O ~
4 ~ ~ ~ ~ ~

Miss!
Player 2's turn to attack Player 1's board.
  0 1 2 3 4
0 ~ ~ ~ ~ ~
1 ~ X X ~ ~
2 ~ ~ O ~ ~
3 ~ ~ ~ ~ ~
4 ~ ~ ~ ~ 